In [16]:
import pandas as pd
import re
from gtfparse import read_gtf
import polars as pl

# Functions

In [17]:
def read_file_and_modify(csv_file, treatment = ''):
    df = pd.read_csv(csv_file)
    if treatment != '':
        df['Treatment'] = treatment

    df['Enhancer'] = df['Enhancer'].str.replace(r'(+)', '')
    return df

# EDA

In [18]:
# Read the files
e20_df = read_file_and_modify('base_data/EnhancerGene20E_df_clean_09_09.csv', '20E')

con_df = read_file_and_modify('base_data/EnhancerGeneCon_df_clean_09_09.csv','Control')

imd_df = read_file_and_modify('base_data/EnhancerGeneHKSM_df_clean_09_09.csv', 'IMD')

combined_df = pd.concat([e20_df, con_df, imd_df])

all_df = read_file_and_modify('base_data/EG_all_clean_09_05.csv')
all_df['Treatment'] = all_df['Treatment'].replace({'HKSM':'IMD'})

tpm_counts = pd.read_csv('base_data/Allsamples_tpm.csv')

In [19]:
print(combined_df.shape)
combined_df.head()

(39130, 10)


,Enhancer,Genes,Cells_20EvsCTRL.table.logFC,coSTARR LogFC 20EvsControl,Immune Process,Time_cluster,Treatment,Control_ave,2021 LogFC IMDvsCTRL,coSTARR LogFC HKSMvs20E
0,2R:16125955-16128021,FBgn0034075,-0.487171,-0.065139,NaN,NaN,20E,NaN,NaN,NaN
1,2R:16125955-16128021,FBgn0034076,0.511375,NaN,NaN,NaN,20E,NaN,NaN,NaN
2,2R:16125955-16128021,FBgn0010052,NaN,-3.331694,NaN,late_C1,20E,NaN,NaN,NaN
3,2R:16125955-16128021,FBgn0050095,NaN,NaN,NaN,NaN,20E,NaN,NaN,NaN
4,2R:16125955-16128021,FBgn0001124,-1.570842,-0.816459,NaN,late_C1,20E,NaN,NaN,NaN


In [20]:
all_df.head()

,Enhancer,Genes,2021 LogFC IMDvsCTRL,coSTARR LogFC HKSMvs20E,Immune Process,Time_cluster,new_act_score,Length,FlybaseID Lem_function,Type_function,...,xrp1_3,TBS_10-3,Treatment,FB_Gene,Control_ave,Cells_20EvsCTRL.table.logFC,coSTARR LogFC 20EvsControl,HKSM_ATAC,Con_ATAC,Accessibility
0,2R:1097242-1098153,FBgn0003256,0.263686928,-0.038253087,Wound repair,0,2182.190,912,[],0,...,1,29,IMD,NaN,NaN,NaN,NaN,yes,yes,Always open
1,2L:517706-518455,FBgn0003963,0.285124923,-0.387774183,Toll; Cellular,0,817.905,750,['FBgn0003963'],Hematopoeisis,...,3,27,IMD,NaN,NaN,NaN,NaN,NaN,NaN,Always closed
2,2L:521029-521849,FBgn0003963,0.285124923,-0.387774183,Toll; Cellular,0,547.332,821,['FBgn0003963'],Hematopoeisis,...,4,16,IMD,NaN,NaN,NaN,NaN,NaN,NaN,Always closed
3,2L:524444-525150,FBgn0003963,0.285124923,-0.387774183,Toll; Cellular,0,1383.430,707,['FBgn0003963'],Hematopoeisis,...,1,20,IMD,NaN,NaN,NaN,NaN,yes,yes,Always open
4,2L:19537144-19537644,FBgn0032798;FBgn0032799;FBgn0041180;FBgn003280...,-0.8434705509999999;1.341826983;1.800378571;0....,0.083930202;0.141035456;0.085920669;-0.2594895...,Toll; Anti-viral;0;Humoral; Cellular,0;early_C2,905.856,501,"['FBgn0003231', 'FBgn0041180']","Phagocytosis, Signaling/Antiviral",...,1,10,IMD,NaN,NaN,NaN,NaN,NaN,NaN,Always closed


In [21]:
print('e20_df.shape',e20_df.shape, '\ncon_df.shape', con_df.shape,'\nimd_df.shape', imd_df.shape)
print()
print("Unique Enhancers in 20E, IMD and Control combined:", len(combined_df['Enhancer'].unique()))
print("Unique Enhancers on collapsed file:", len(all_df['Enhancer'].unique()))


e20_df.shape (14154, 7) 
con_df.shape (11272, 6) 
imd_df.shape (13704, 7)

Unique Enhancers in 20E, IMD and Control combined: 8204
Unique Enhancers on collapsed file: 8204


# Enhancer Table

In [22]:
# Extract the 'Enhancer' column
enhancer = combined_df['Enhancer']

# Parse chromosome, start, and end from the Enhancer string
combined_df['Chromosome'] = enhancer.apply(lambda x: x.split(':')[0])
combined_df['Start'] = enhancer.apply(lambda x: int(re.search(r':(\d+)-', x).group(1)))
combined_df['End'] = enhancer.apply(lambda x: int(re.search(r'-(\d+)', x).group(1)))
combined_df['Length'] = abs(combined_df['End'] - combined_df['Start'])

In [23]:
# Display the updated DataFrame and store it to csv
combined_df.reset_index(drop = True, inplace = True)
tf_col = ['crp_3', 'EcR_usp_3', 'Eip74EF_3', 'gcm_3', 'Hnf4_3',
       'kay_Jra_3', 'Rel_3', 'slp2_fork_3', 'SREBP_3', 'srp_SANGER_3', 'Trl_3', 'XBP1_3', 'xrp1_3']
all_df['TF_counts'] = all_df[tf_col].apply(lambda row: '; '.
                                             join([f'{col}: {int(row[col])}' for col in tf_col if row[col] > 0]),
                                             axis = 1)

In [24]:
enhancer_df = pd.merge(
    combined_df[['Enhancer','Chromosome','Start','End','Length','Treatment']],
    all_df[['Enhancer','TF_counts','TBS_10-3']],
    on='Enhancer',
    how='left'
).drop_duplicates(ignore_index=True)

enhancer_df = enhancer_df.rename(columns={
    'TBS_10-3':'TBS'
})

enhancer_df.to_csv('processed_data/enhancer.csv', index=False)

# Genes Table

In [25]:
# gene csv
gene_df = combined_df[['Genes','Immune Process','Time_cluster']]
gene_df.reset_index(drop = True, inplace = True)
gene_df.rename(columns = {'Genes':'gene_id'},inplace = True)
gene_df

/var/folders/mh/p8rsw7z90bv6kv8r8qnf4tnr0000gn/T/ipykernel_20199/3327782363.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df.rename(columns = {'Genes':'gene_id'},inplace = True)


,gene_id,Immune Process,Time_cluster
0,FBgn0034075,NaN,NaN
1,FBgn0034076,NaN,NaN
2,FBgn0010052,NaN,late_C1
3,FBgn0050095,NaN,NaN
4,FBgn0001124,NaN,late_C1
...,...,...,...
39125,FBgn0267432,NaN,NaN
39126,FBgn0001313,NaN,NaN
39127,NaN,NaN,NaN
39128,NaN,NaN,NaN


In [26]:
# Make a unique list of genes
unique_genes = gene_df['gene_id'].dropna().unique()
unique_genes = unique_genes.tolist()
len(unique_genes)

10411

In [27]:
# Read gtf file
gtf_file = read_gtf('base_data/Drosophila_melanogaster.BDGP6.54.115.gtf')
print(gtf_file.shape)
gtf_file.head()

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_source', 'transcript_biotype', 'tag', 'exon_number', 'exon_id', 'transcript_name', 'protein_id']


(567804, 20)


seqname,source,feature,start,end,score,strand,frame,gene_id,gene_name,gene_source,gene_biotype,transcript_id,transcript_source,transcript_biotype,tag,exon_number,exon_id,transcript_name,protein_id
cat,cat,cat,i64,i64,f32,cat,i64,str,str,str,str,str,str,str,str,str,str,str,str
"""3R""","""FlyBase""","""gene""",10964778,10964818,null,"""+""",0,"""FBti0060911""","""transib2{}2850""","""FlyBase""","""transposable_element""","""""","""""","""""","""""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""transcript""",10964778,10964818,null,"""+""",0,"""FBti0060911""","""transib2{}2850""","""FlyBase""","""transposable_element""","""FBti0060911-RA""","""FlyBase""","""transposable_element""","""Ensembl_canonical""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""exon""",10964778,10964818,null,"""+""",0,"""FBti0060911""","""transib2{}2850""","""FlyBase""","""transposable_element""","""FBti0060911-RA""","""FlyBase""","""transposable_element""","""Ensembl_canonical""","""1""","""FBti0060911-RA-E1""","""""",""""""
"""3R""","""FlyBase""","""gene""",13944654,13945812,null,"""-""",0,"""FBgn0038189""","""Art6""","""FlyBase""","""protein_coding""","""""","""""","""""","""""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""transcript""",13944654,13945812,null,"""-""",0,"""FBgn0038189""","""Art6""","""FlyBase""","""protein_coding""","""FBtr0082878""","""FlyBase""","""protein_coding""","""Ensembl_canonical""","""""","""""","""Art6-RA""",""""""


In [28]:
gtf_genes = gtf_file.filter(pl.col("feature") == "gene")
gtf_genes.head(n = 10)

seqname,source,feature,start,end,score,strand,frame,gene_id,gene_name,gene_source,gene_biotype,transcript_id,transcript_source,transcript_biotype,tag,exon_number,exon_id,transcript_name,protein_id
cat,cat,cat,i64,i64,f32,cat,i64,str,str,str,str,str,str,str,str,str,str,str,str
"""3R""","""FlyBase""","""gene""",10964778,10964818,null,"""+""",0,"""FBti0060911""","""transib2{}2850""","""FlyBase""","""transposable_element""","""""","""""","""""","""""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""gene""",13944654,13945812,null,"""-""",0,"""FBgn0038189""","""Art6""","""FlyBase""","""protein_coding""","""""","""""","""""","""""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""gene""",8359958,8362554,null,"""-""",0,"""FBgn0037583""","""Veneno""","""FlyBase""","""protein_coding""","""""","""""","""""","""""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""gene""",18330932,18360728,null,"""-""",0,"""FBgn0038606""","""CG15803""","""FlyBase""","""protein_coding""","""""","""""","""""","""""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""gene""",17703695,17705581,null,"""+""",0,"""FBgn0267702""","""lncRNA:CR46035""","""FlyBase""","""ncRNA""","""""","""""","""""","""""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""gene""",5862865,5959555,null,"""-""",0,"""FBgn0083949""","""side-III""","""FlyBase""","""protein_coding""","""""","""""","""""","""""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""gene""",12452464,12453196,null,"""+""",0,"""FBgn0038054""","""CG5509""","""FlyBase""","""protein_coding""","""""","""""","""""","""""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""gene""",9483378,9484078,null,"""-""",0,"""FBgn0266730""","""lncRNA:CR45203""","""FlyBase""","""ncRNA""","""""","""""","""""","""""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""gene""",4639789,4640004,null,"""-""",0,"""FBgn0286036""","""sisRNA:CR46358""","""FlyBase""","""ncRNA""","""""","""""","""""","""""","""""","""""","""""",""""""


In [29]:
filtered_genes = gtf_genes.filter(pl.col("gene_id").is_in(unique_genes))

In [30]:
result_df = pd.merge(
    filtered_genes[['gene_id','seqname','start','end','gene_name']].to_pandas(),
    gene_df[['gene_id','Immune Process', 'Time_cluster']],
    on='gene_id',
    how='left')

result_df.columns = ['GeneID','Chromosome', 'Start', 'End', 'GeneName','Immune Process','Time_cluster']
result_df['Length'] = abs(result_df['End'] - result_df['Start'])

result_df.drop_duplicates(ignore_index = True,inplace = True)
result_df.reset_index(drop=True, inplace=True)
result_df


,GeneID,Chromosome,Start,End,GeneName,Immune Process,Time_cluster,Length
0,FBgn0038189,3R,13944654,13945812,Art6,NaN,NaN,1158
1,FBgn0037583,3R,8359958,8362554,Veneno,NaN,NaN,2596
2,FBgn0267702,3R,17703695,17705581,lncRNA:CR46035,NaN,NaN,1886
3,FBgn0083949,3R,5862865,5959555,side-III,NaN,NaN,96690
4,FBgn0038054,3R,12452464,12453196,CG5509,NaN,NaN,732
...,...,...,...,...,...,...,...,...
10278,FBgn0267880,Y,3113685,3114094,CR46168,NaN,NaN,409
10279,FBgn0039925,4,1147466,1169239,Kif3C,NaN,NaN,21773
10280,FBgn0039924,4,1192419,1196848,PIP4K,NaN,NaN,4429
10281,FBgn0250819,4,1185768,1192985,CG33521,NaN,NaN,7217


In [31]:
# Convert unique genes list to a set
unique_genes_set = set(unique_genes)

# Convert filtered gene_ids (Polars) to a set
filtered_gene_ids_set = set(filtered_genes["gene_id"].to_list())

# Find unmapped genes
unmapped_genes = unique_genes_set - filtered_gene_ids_set

print(f"Number of unmapped genes: {len(unmapped_genes)}")
unmapped_genes

Number of unmapped genes: 129


{'FBgn0000054',
 'FBgn0000409',
 'FBgn0000497',
 'FBgn0000556',
 'FBgn0001974',
 'FBgn0001981',
 'FBgn0002607',
 'FBgn0003328',
 'FBgn0003732',
 'FBgn0003887',
 'FBgn0003888',
 'FBgn0003926',
 'FBgn0004009',
 'FBgn0004364',
 'FBgn0005592',
 'FBgn0010575',
 'FBgn0011241',
 'FBgn0011655',
 'FBgn0014002',
 'FBgn0015393',
 'FBgn0016700',
 'FBgn0019643',
 'FBgn0019886',
 'FBgn0019947',
 'FBgn0021818',
 'FBgn0022772',
 'FBgn0024332',
 'FBgn0025286',
 'FBgn0025683',
 'FBgn0025687',
 'FBgn0027585',
 'FBgn0028336',
 'FBgn0028519',
 'FBgn0028665',
 'FBgn0029082',
 'FBgn0029173',
 'FBgn0029656',
 'FBgn0029707',
 'FBgn0029899',
 'FBgn0030170',
 'FBgn0030294',
 'FBgn0031821',
 'FBgn0031842',
 'FBgn0032276',
 'FBgn0032679',
 'FBgn0032906',
 'FBgn0032915',
 'FBgn0033059',
 'FBgn0033359',
 'FBgn0033465',
 'FBgn0033981',
 'FBgn0034496',
 'FBgn0034967',
 'FBgn0035707',
 'FBgn0035987',
 'FBgn0036169',
 'FBgn0036660',
 'FBgn0037010',
 'FBgn0037417',
 'FBgn0037636',
 'FBgn0038035',
 'FBgn0038052',
 'FBgn00

In [32]:
gtf_file

seqname,source,feature,start,end,score,strand,frame,gene_id,gene_name,gene_source,gene_biotype,transcript_id,transcript_source,transcript_biotype,tag,exon_number,exon_id,transcript_name,protein_id
cat,cat,cat,i64,i64,f32,cat,i64,str,str,str,str,str,str,str,str,str,str,str,str
"""3R""","""FlyBase""","""gene""",10964778,10964818,null,"""+""",0,"""FBti0060911""","""transib2{}2850""","""FlyBase""","""transposable_element""","""""","""""","""""","""""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""transcript""",10964778,10964818,null,"""+""",0,"""FBti0060911""","""transib2{}2850""","""FlyBase""","""transposable_element""","""FBti0060911-RA""","""FlyBase""","""transposable_element""","""Ensembl_canonical""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""exon""",10964778,10964818,null,"""+""",0,"""FBti0060911""","""transib2{}2850""","""FlyBase""","""transposable_element""","""FBti0060911-RA""","""FlyBase""","""transposable_element""","""Ensembl_canonical""","""1""","""FBti0060911-RA-E1""","""""",""""""
"""3R""","""FlyBase""","""gene""",13944654,13945812,null,"""-""",0,"""FBgn0038189""","""Art6""","""FlyBase""","""protein_coding""","""""","""""","""""","""""","""""","""""","""""",""""""
"""3R""","""FlyBase""","""transcript""",13944654,13945812,null,"""-""",0,"""FBgn0038189""","""Art6""","""FlyBase""","""protein_coding""","""FBtr0082878""","""FlyBase""","""protein_coding""","""Ensembl_canonical""","""""","""""","""Art6-RA""",""""""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""211000022279165""","""FlyBase""","""transcript""",14,1118,null,"""-""",0,"""FBgn0259870""","""Su(Ste):CR42439""","""FlyBase""","""ncRNA""","""FBtr0300167""","""FlyBase""","""ncRNA""","""Ensembl_canonical""","""""","""""","""Su(Ste):CR42439-RA""",""""""
"""211000022279165""","""FlyBase""","""exon""",14,1118,null,"""-""",0,"""FBgn0259870""","""Su(Ste):CR42439""","""FlyBase""","""ncRNA""","""FBtr0300167""","""FlyBase""","""ncRNA""","""Ensembl_canonical""","""1""","""FBtr0300167-E1""","""Su(Ste):CR42439-RA""",""""""
"""211000022279264""","""FlyBase""","""gene""",180,614,null,"""-""",0,"""FBgn0085511""","""lncRNA:CR40719""","""FlyBase""","""ncRNA""","""""","""""","""""","""""","""""","""""","""""",""""""


In [33]:
unmapped_feature_mapping = gtf_file.filter(pl.col("gene_id").is_in(unmapped_genes))
unmapped_feature_mapping

# Unmapped genes do not map to anything, but added to the final genes.csv

seqname,source,feature,start,end,score,strand,frame,gene_id,gene_name,gene_source,gene_biotype,transcript_id,transcript_source,transcript_biotype,tag,exon_number,exon_id,transcript_name,protein_id
cat,cat,cat,i64,i64,f32,cat,i64,str,str,str,str,str,str,str,str,str,str,str,str


In [34]:
unmapped_genes = pd.DataFrame(unmapped_genes,columns=['gene_id']).rename(columns={'gene_id':'GeneID'})
genes = pd.concat([result_df,unmapped_genes])
print(len(unique_genes),len(genes))

10411 10412


## TPM


~~S1: Control, Replicate 1~~ - Outlier

S2: 20E, Replicate 1

S3: IMD (HKSM), Replicate 1

S4: Control, Replicate 2

S5: 20E, Replicate 2

S6: IMD (HKSM), Replicate 2

S7: Control, Replicate 3

S8: 20E, Replicate 3

S9: IMD (HKSM), Replicate 3

In [35]:
# Control Average
tpm_counts['tpm_ctrl'] = (tpm_counts['S4_counts'] + tpm_counts['S7_counts'])/2

# 20E average
tpm_counts['tpm_20e'] = (tpm_counts['S2_counts'] + tpm_counts['S5_counts'] + tpm_counts['S8_counts'])/3

# IMD average
tpm_counts['tpm_imd'] = (tpm_counts['S3_counts'] + tpm_counts['S6_counts'] + tpm_counts['S9_counts'])/3
tpm_counts

,FB_Gene,S1_counts,S2_counts,S3_counts,S4_counts,S5_counts,S6_counts,S7_counts,S8_counts,S9_counts,tpm_ctrl,tpm_20e,tpm_imd
0,FBgn0031081,3.557237,0.010032,0.027937,0.000000,0.021195,0.000000,0.000000,0.136597,0.000000,0.000000,0.055941,0.009312
1,FBgn0052826,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,FBgn0031085,5.224219,0.000000,0.110539,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036846
3,FBgn0062565,59.890682,113.678741,106.025847,50.292973,84.532776,66.951918,36.736645,72.550286,52.186025,43.514809,90.253934,75.054597
4,FBgn0031088,1.449757,0.015499,0.014386,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005166,0.004795
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17601,FBgn0031288,0.000000,0.123413,0.190928,0.349307,0.130369,0.218569,0.900850,0.373418,0.160965,0.625079,0.209066,0.190154
17602,FBgn0031289,0.000000,0.082136,0.076242,0.697436,0.260297,0.130920,0.449665,0.310656,0.000000,0.573550,0.217697,0.069054
17603,FBgn0002936,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
17604,FBgn0002563,0.173298,0.000000,0.016433,0.125267,0.000000,0.000000,0.048459,0.000000,0.000000,0.086863,0.000000,0.005478


In [36]:
genes

,GeneID,Chromosome,Start,End,GeneName,Immune Process,Time_cluster,Length
0,FBgn0038189,3R,13944654.0,13945812.0,Art6,NaN,NaN,1158.0
1,FBgn0037583,3R,8359958.0,8362554.0,Veneno,NaN,NaN,2596.0
2,FBgn0267702,3R,17703695.0,17705581.0,lncRNA:CR46035,NaN,NaN,1886.0
3,FBgn0083949,3R,5862865.0,5959555.0,side-III,NaN,NaN,96690.0
4,FBgn0038054,3R,12452464.0,12453196.0,CG5509,NaN,NaN,732.0
...,...,...,...,...,...,...,...,...
124,FBgn0036169,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,FBgn0086037,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,FBgn0086052,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,FBgn0025687,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
genes = pd.merge(genes,tpm_counts[['FB_Gene','tpm_ctrl','tpm_20e','tpm_imd']],left_on='GeneID',right_on='FB_Gene',how='left')
genes = genes.round(decimals=5)
genes

,GeneID,Chromosome,Start,End,GeneName,Immune Process,Time_cluster,Length,FB_Gene,tpm_ctrl,tpm_20e,tpm_imd
0,FBgn0038189,3R,13944654.0,13945812.0,Art6,NaN,NaN,1158.0,FBgn0038189,0.00000,0.00000,0.00000
1,FBgn0037583,3R,8359958.0,8362554.0,Veneno,NaN,NaN,2596.0,FBgn0037583,47.17868,47.39516,30.42884
2,FBgn0267702,3R,17703695.0,17705581.0,lncRNA:CR46035,NaN,NaN,1886.0,FBgn0267702,1.19688,3.21491,1.97773
3,FBgn0083949,3R,5862865.0,5959555.0,side-III,NaN,NaN,96690.0,FBgn0083949,0.03985,0.05868,0.03951
4,FBgn0038054,3R,12452464.0,12453196.0,CG5509,NaN,NaN,732.0,FBgn0038054,1.33923,1.78048,0.87884
...,...,...,...,...,...,...,...,...,...,...,...,...
10407,FBgn0036169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10408,FBgn0086037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FBgn0086037,0.49338,0.00000,0.33462
10409,FBgn0086052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FBgn0086052,0.00000,0.00000,0.15685
10410,FBgn0025687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
genes.drop('FB_Gene',axis=1).to_csv('processed_data/genes.csv', index = False)

### flybase gtf

In [39]:
'''# Read gtf file
gtf_file = read_gtf('base_data/dmel-all-r6.64.gtf')
print(gtf_file.shape)
gtf_file.head()'''

"# Read gtf file\ngtf_file = read_gtf('base_data/dmel-all-r6.64.gtf')\nprint(gtf_file.shape)\ngtf_file.head()"

In [40]:
'''gtf_genes = gtf_file.filter(pl.col("feature") == "gene")
gtf_genes.head(n = 10)'''

'gtf_genes = gtf_file.filter(pl.col("feature") == "gene")\ngtf_genes.head(n = 10)'

In [41]:
'''filtered_genes = gtf_genes.filter(pl.col("gene_id").is_in(unique_genes))'''

'filtered_genes = gtf_genes.filter(pl.col("gene_id").is_in(unique_genes))'

In [42]:
'''result_df = pd.merge(
    filtered_genes[['gene_id','seqname','start','end','gene_symbol']].to_pandas(),
    gene_df[['gene_id','Immune Process', 'Time_cluster']],
    on='gene_id',
    how='left'
)
result_df.columns = ['GeneID','Chromosome', 'Start', 'End', 'GeneName','Immune Process','Time_cluster']
result_df.drop_duplicates(ignore_index = True,inplace = True)
result_df.reset_index(drop=True, inplace=True)
result_df
'''

"result_df = pd.merge(\n    filtered_genes[['gene_id','seqname','start','end','gene_symbol']].to_pandas(),\n    gene_df[['gene_id','Immune Process', 'Time_cluster']],\n    on='gene_id',\n    how='left'\n)\nresult_df.columns = ['GeneID','Chromosome', 'Start', 'End', 'GeneName','Immune Process','Time_cluster']\nresult_df.drop_duplicates(ignore_index = True,inplace = True)\nresult_df.reset_index(drop=True, inplace=True)\nresult_df\n"

In [43]:
'''result_df.to_csv('genes.csv', index = False)'''

"result_df.to_csv('genes.csv', index = False)"

In [44]:
'''# Convert unique genes list to a set
unique_genes_set = set(unique_genes)

# Convert filtered gene_ids (Polars) to a set
filtered_gene_ids_set = set(filtered_genes["gene_id"].to_list())

# Find unmapped genes
unmapped_genes = unique_genes_set - filtered_gene_ids_set

print(f"Number of unmapped genes: {len(unmapped_genes)}")
unmapped_genes

# Unmapped genes were 140 (ensembl had 129 unmapped genes)
Hence, ensembl is the better ref database here'''

'# Convert unique genes list to a set\nunique_genes_set = set(unique_genes)\n\n# Convert filtered gene_ids (Polars) to a set\nfiltered_gene_ids_set = set(filtered_genes["gene_id"].to_list())\n\n# Find unmapped genes\nunmapped_genes = unique_genes_set - filtered_gene_ids_set\n\nprint(f"Number of unmapped genes: {len(unmapped_genes)}")\nunmapped_genes\n\n# Unmapped genes were 140 (ensembl had 129 unmapped genes)\nHence, ensembl is the better ref database here'

# Associations Table

In [45]:
print(combined_df.columns)
print(all_df.columns)

Index(['Enhancer', 'Genes', 'Cells_20EvsCTRL.table.logFC',
       'coSTARR LogFC 20EvsControl', 'Immune Process', 'Time_cluster',
       'Treatment', 'Control_ave', '2021 LogFC IMDvsCTRL',
       'coSTARR LogFC HKSMvs20E', 'Chromosome', 'Start', 'End', 'Length'],
      dtype='object')
Index(['Enhancer', 'Genes', '2021 LogFC IMDvsCTRL', 'coSTARR LogFC HKSMvs20E',
       'Immune Process', 'Time_cluster', 'new_act_score', 'Length',
       'FlybaseID Lem_function', 'Type_function', 'FlybaseID Lem_subtype',
       'subtype', 'crp_3', 'EcR_usp_3', 'Eip74EF_3', 'gcm_3', 'Hnf4_3',
       'kay_Jra_3', 'Rel_3', 'slp2_fork_3', 'SREBP_3', 'srp_SANGER_3', 'Trl_3',
       'XBP1_3', 'xrp1_3', 'TBS_10-3', 'Treatment', 'FB_Gene', 'Control_ave',
       'Cells_20EvsCTRL.table.logFC', 'coSTARR LogFC 20EvsControl',
       'HKSM_ATAC', 'Con_ATAC', 'Accessibility', 'TF_counts'],
      dtype='object')


In [46]:
combined_df.shape

(39130, 14)

In [47]:
print(combined_df['Treatment'].unique())
all_df['Treatment'].unique()

['20E' 'Control' 'IMD']


array(['IMD', 'Control', '20E'], dtype=object)

In [48]:
''' # FROM PREVIOUS CODE
# USED AS REFERENCE
asc_df = all_df[['Enhancer','Genes','2021 LogFC IMDvsCTRL','Cells_20EvsCTRL.table.logFC','coSTARR LogFC HKSMvs20E','coSTARR LogFC 20EvsControl', 'Origin','Activity_score']]'''

asc_df = pd.merge(
    combined_df[['Enhancer','Genes','coSTARR LogFC HKSMvs20E',
                 'coSTARR LogFC 20EvsControl','2021 LogFC IMDvsCTRL','Treatment']],
    all_df[['Enhancer', 'Treatment', 'new_act_score','Accessibility']],
    on=['Enhancer','Treatment'],
    how='left'
)

In [49]:
asc_df[[]].notna()

asc_df = asc_df.rename(columns={'Genes':'Gene'})

# Drop rows where ALL logFC & Gene are missing
asc_df = asc_df[~asc_df[['Gene',
                         'coSTARR LogFC HKSMvs20E',
                         '2021 LogFC IMDvsCTRL',
                         'coSTARR LogFC 20EvsControl']].isnull().all(axis=1)]

In [50]:
asc_df

,Enhancer,Gene,coSTARR LogFC HKSMvs20E,coSTARR LogFC 20EvsControl,2021 LogFC IMDvsCTRL,Treatment,new_act_score,Accessibility
0,2R:16125955-16128021,FBgn0034075,NaN,-0.065139,NaN,20E,1344.89,Always open
1,2R:16125955-16128021,FBgn0034076,NaN,NaN,NaN,20E,1344.89,Always open
2,2R:16125955-16128021,FBgn0010052,NaN,-3.331694,NaN,20E,1344.89,Always open
3,2R:16125955-16128021,FBgn0050095,NaN,NaN,NaN,20E,1344.89,Always open
4,2R:16125955-16128021,FBgn0001124,NaN,-0.816459,NaN,20E,1344.89,Always open
...,...,...,...,...,...,...,...,...
39102,Y:14211-14771,FBgn0267433,NaN,NaN,1.133805,IMD,1424.51,Always closed
39104,Y:16815-17445,FBgn0267433,NaN,NaN,1.133805,IMD,1062.99,Always closed
39116,Y:283890-284788,FBgn0267449,NaN,NaN,NaN,IMD,1250.94,Always closed
39125,Y:49440-50073,FBgn0267432,NaN,NaN,NaN,IMD,1134.60,Always closed


In [51]:
asc_df = asc_df.drop_duplicates().reset_index(drop=True)
asc_df = asc_df.round(5)

asc_df.to_csv('processed_data/associations.csv', index=False)

In [52]:
asc_df.columns

Index(['Enhancer', 'Gene', 'coSTARR LogFC HKSMvs20E',
       'coSTARR LogFC 20EvsControl', '2021 LogFC IMDvsCTRL', 'Treatment',
       'new_act_score', 'Accessibility'],
      dtype='object')

# Tab 3 data

In [53]:
association_df = pd.read_csv('base_data/EG_activity_class_modified.csv')

In [54]:
association_df.drop(columns=['Unnamed: 7', 'Unnamed: 8'], inplace=True)
association_df.columns

Index(['Enhancer', 'Activity class', 'Accessibility', 'Gene',
       'Distance to enhancer', 'Time Cluster', 'Broad Immune Role'],
      dtype='object')

In [55]:
association_df.nunique()

Enhancer                4270
Activity class             7
Accessibility              4
Gene                    4640
Distance to enhancer    6381
Time Cluster               4
Broad Immune Role          7
dtype: int64

In [56]:
for col in association_df.columns:
    l = association_df[col].unique()
    if len(l) < 50:
        print(col, l)

Activity class ['Control' '20E' 'HKSM' 'Control + 20E' 'Control + HKSM' 'HKSM + 20E'
 'Constitutive']
Accessibility ['Always open' 'Always closed' 'HKSM closed' 'HKSM opened']
Time Cluster [nan 'Early' 'Mid' 'Late' 'Midlate']
Broad Immune Role [nan 'Signaling' 'Recognition' 'Other' 'Effector' 'Effector (AMP)'
 'Anti-viral' 'Effector (ROS)']


In [57]:
association_df.value_counts(subset=['Enhancer','Gene'],ascending=False)

Enhancer              Gene       
2L:10056850-10057876  FBgn0032178    1
3R:21662327-21662909  FBgn0011766    1
                      FBgn0259113    1
3R:2168891-2169514    .              1
3R:21731337-21731939  FBgn0038897    1
                                    ..
2R:7457211-7458216    FBgn0033178    1
2R:7492719-7493388    FBgn0015929    1
                      FBgn0033183    1
                      FBgn0259745    1
Y:922754-923487       .              1
Name: count, Length: 9692, dtype: int64

In [58]:
association_df.to_csv('processed_data/tab3_data.csv', index=False)